# Práctica 2. Aprendizaje por refuerzo
## Inteligencia Artificial I    2021/2022

## Parte 1. Taxi
En este ejemplo (visto en clase) hay 4 ubicaciones (etiquetadas con letras diferentes) y nuestro trabajo es recoger al pasajero en una ubicación y dejarlo en otra. Se trata de llevar al pasajero que inicialmente está en (Y) a la posición destino (R). El taxi solo puede coger y dejar pasajeros en las posiciones marcadas.
Recibimos +20 puntos por dejar al pasajero con éxito y perderemos 1 punto por cada paso de tiempo. También hay una penalización de 10 puntos por acciones ilegales de recoger y dejar.

Vamos a utilizar OpenAI's Gym en Python donde tenemos definido este entorno y podemos desarrollar nuestro agente y evaluarlo. gym nos proporciona la representación y la visualización del tablero (render) por lo que no tenemos que hacerlo (aunque sería sencillo representarlo y resolverlo con cualquier algoritmo de búsqueda en AIMA). 

In [ ]:
# Para el ejemplo del taxi y el cart pool es necesario instalar algunas librerías (gym,cmake,scipy) si no están ya instaladas
import sys
!{sys.executable} -m pip install cmake
# Gym is a toolkit for developing and comparing reinforcement learning algorithms. It makes no assumptions about the structure of your agent, and is compatible with any numerical computation library, such as TensorFlow or Theano.
# La librería gym proporciona problemas de prueba — environments — con una interfaz general que puedes usar para probar 
# distintos algoritmos y configuraciones de RL. 

!{sys.executable} -m pip install gym
#tps://gym.openai.com/docs/

!{sys.executable} -m pip install scipy

![image.png](attachment:image.png)

In [ ]:
import gym
# el ejemplo del taxi es un entorno que ya está definido en gym por lo que no tenemos que representar este problema.
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



La interfaz principal del entorno gym es env. 
Vamos a utilizar los siguientes métodos de env:

- env.reset: restablece el entorno y devuelve un estado inicial aleatorio.

- env.step (acción): realiza un paso en el entorno. Devuelve:

      observación: Observaciones del entorno
      recompensa: si su acción fue beneficiosa o no
      done: Indica si hemos recogido y dejado a un pasajero (fin de un episodio)
      info: información adicional como rendimiento y latencia para depuración
      
- env.render: renderiza el entorno (útil para visualizar el entorno)

In [ ]:
env.reset() # resetea el estado del problema a un estado aleatorio. 

266

In [ ]:
env.render() #visualiza el estado del problema

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



- El cuadrado representa el taxi, que es amarillo sin pasajero y verde con pasajero.
- La marca ("|") representa una pared que el taxi no puede cruzar.
- R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra azul representa la ubicación actual de recogida de pasajeros y la letra morada es el destino actual.


In [ ]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


El Action Space tiene tamaño 6 y el State Space tiene tamaño 500. El algoritmo RL no necesita más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de forma única asignando un número único a cada estado posible, y RL aprende a elegir un número de acción de 0 a 5 donde:
   - 0 = sur
   - 1 = norte
   - 2 = este
   - 3 = oeste
   - 4 = recoger
   - 5 = dejar

Los 500 estados corresponden a una codificación de la ubicación del taxi, la ubicación del pasajero y la ubicación de destino.

El aprendizaje por refuerzo aprenderá un mapeo de estados con la acción óptima a realizar en ese estado. Para ello realizaremos un proceso de exploración, es decir, el agente explora el entorno y toma acciones basadas en las recompensas definidas en el entorno. La acción óptima para cada estado es la acción que tiene la mayor recompensa acumulativa a largo plazo según la fórmula vista para el Q-learning.

Dado el estado representado en la imagen anterior, vamos a codificar su estado y dárselo al entorno para que se renderice en Gym. 
Las filas y columnas se numeran de 0 a 4 y, como se ve en la imagen, tenemos el taxi en la fila 3, columna 1, nuestro pasajero está en la ubicación 2 y nuestro destino es la ubicación 0:  R (0), G (1), Y (2), B (3)

Usando el método de codificación de estado Taxi-v3, podemos hacer lo siguiente:

In [ ]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state
env.render()


State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Podemos asertar las posiciones en el estado con env.encode() o usar los números entre 0 y 499 que son los 500 estados válidos en el espacio de estados. 

In [ ]:
env.s = 499
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+



In [ ]:
env.s = 0
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [ ]:
# Con el diccionario P del entorno (tiene más informacion que la matriz R que hemos visto en clase) podemos ver los valores de reward asignados por defecto al estado 328 que usamos como ejemplo
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

Este diccionario tiene la siguiente estructura {acción: [(probabilidad, próximo estado, recompensa, hecho)]}.

- El valor de 0-5 corresponde a las acciones (sur, norte, este, oeste, recogida, entrega) que el taxi puede realizar.
- La probabilidad es siempre 1.0 (en este entorno)
- nextstate muestra el estado siguiente usando la acción en este índice del diccionario.
- Las acciones de movimiento tienen por defecto una recompensa de -1 y pickup / dropoff tiene -10 Si estuvimos en un estado donde el taxi tiene un pasajero y estamos en la posición destino correcta se vería una recompensa de 20 en la action dropoff (5 ))
- done se utiliza para indicarnos cuándo hemos dejado a un pasajero en el lugar correcto. Cada dejada de un pasajero (dropoff) con éxito es el final de un episodio.

Hay que tener en cuenta que si nuestro agente eligiera explorar la acción tres (3) en este estado, iría hacia el oeste y chocaría contra una pared. El código fuente ha hecho imposible mover el taxi a través de una pared, por lo que si el taxi elige esa acción, seguirá acumulando -1 penalizaciones, lo que afecta la recompensa a largo plazo.

![image.png](attachment:image.png)

### Paso 1. Resolvemos el problema sin aprendizaje por refuerzo

Veamos qué pasaría si intentamos utilizar la fuerza bruta para resolver el problema sin RL.
Dado que tenemos nuestra tabla P para las recompensas predeterminadas en cada estado, podemos intentar que nuestro taxi navegue solo con eso.
Crearemos un bucle infinito que se ejecutará hasta que un pasajero llegue a un destino (un episodio), o en otras palabras, cuando la recompensa recibida sea 20.

El método env.action_space.sample () selecciona automáticamente una acción aleatoria del conjunto de todas las acciones posibles.


In [ ]:
env.s = 328  # el estado de la imagen

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))


Timesteps taken: 304
Penalties incurred: 92


In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        #print(frame['frame'].getvalue())
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

In [ ]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 304
State: 0
Action: 5
Reward: 20


El agente va a ciegas y utiliza miles de pasos de tiempo y realiza muchos drop-offs incorrectos para entregar un solo pasajero al destino correcto (cuando acierte).  

Dejamos como ejercicio opcional la resolución de este problema con cualquiera de los algoritmos de búsqueda de AIMA definiendo la clase Problem usando la codificación numérica de los estados de este entorno y las 5 acciones que hemos visto haciendo una llamada a next_state, reward, done, info = env.step(action) 

Vamos a resolverlo aquí con Aprendizaje por refuerzo que empieza haciendo ciclos a ciegas y aprende de la experiencia pasada.  Cuando "por casualidad" acertamos el agente guarda en la memoria (en forma de recompensa) qué acción fue la mejor para cada estado. Así en el futuro elegirá esa acción. 

In [ ]:
import gym
env = gym.make("Taxi-v3").env
# Estado inicial aleatorio
env.reset() # reset environment to a new, random state   EStado inicial aleatorio 
# O estado inicial establecido 
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [ ]:
env.s = 499
env.render()
# cuadrado amarillo: taxi sin pasajero and green with a passenger.
# R, G, Y, B are the possible pickup and destination locations. 
#The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+



In [ ]:
state = env.encode(3, 1, 2, 0)     # 3,1 son as coordenadas del taxi.  
# 2 y 0 son las posiciones del pasajero y dejada del pasajero que estan numeradas de 0..4
#  passenger index, destination index  (creo que para indicar que esta en el taxi se usa tambien .. mirar.)

action = env.action_space.sample()    # genera una acción aleatoria
# Recordamos las acciones: 0 = south; 1 = north; 2 = east; 3 = west; 4 = pickup; 5 = dropoff
env.render()
next_state, reward, done, info = env.step(action) 
print("State:", state)
print("Action:", action)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+

State: 328
Action: 0


In [ ]:
#next_state = env.step(action)[0] 
print("Next State:", next_state)
env.s = next_state
env.render()

Next State: 499
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (South)


In [ ]:
action = 5   
# Recordamos las acciones: 0 = south; 1 = north; 2 = east; 3 = west; 4 = pickup; 5 = dropoff
env.render()
next_state, reward, done, info = env.step(action) 
print("State:", state)
print("Next Action:", action)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (South)
State: 328
Next Action: 5


In [ ]:
print("Next State:", next_state)
env.s = next_state
env.render()
# como hacemos dropoff de un taxi vacio no hace nada

Next State: 499
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (Dropoff)


In [ ]:
env.s = 300
state = env.s
next_state = 0
action = 3   
env.render()
next_state, reward, done, info = env.step(action) 
print("State:", state)
print("Next Action:", action)
print("Next State:", next_state)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
State: 300
Next Action: 3
Next State: 300


In [ ]:
env.s = next_state
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


### Aprendizaje por refuerzo

Vamos a utilizar el algoritmo de Q-learning que hemos visto en clase y que le dará a nuestro agente algo de memoria.
Básicamente, Q-learning permite al agente utilizar las recompensas del entorno para aprender, con el tiempo, la mejor acción a realizar en un estado determinado.
En nuestro entorno de Taxi, tenemos la tabla de recompensas, P, de la que el agente aprenderá. Lo hace buscando recibir una recompensa por realizar una acción en el estado actual y luego actualizar un valor Q para recordar si esa acción fue beneficiosa.
Los valores almacenados en la tabla Q se denominan valores Q y se asignan a una combinación (estado, acción).
Un valor Q para una combinación de acción de estado particular es representativo de la "calidad" de una acción tomada desde ese estado. Mejores valores Q implican mejores posibilidades de obtener mayores recompensas.
Por ejemplo, si el taxi se enfrenta a un estado que incluye a un pasajero en su ubicación actual, es muy probable que el valor Q para la recogida sea más alto en comparación con otras acciones, como la bajada o el norte.
Los valores Q se inicializan a un valor arbitrario y, a medida que el agente se expone al entorno y recibe diferentes recompensas al ejecutar diferentes acciones, los valores Q se actualizan mediante la ecuación:

Q (estado, acción) ← (1 − α) Q (estado, acción) + α (recompensa + γ maxa Q (siguiente estado, todas las acciones))

Dónde:
- α (alfa) es la tasa de aprendizaje (0 <α≤1) - Al igual que en los entornos de aprendizaje supervisado, αα es la medida en que nuestros valores Q se actualizan en cada iteración.
- γ (gamma) es el factor de descuento (0≤γ≤1) - determina cuánta importancia queremos dar a las recompensas futuras. Un valor alto para el factor de descuento (cercano a 1) captura la recompensa efectiva a largo plazo, mientras que un factor de descuento de 0 hace que nuestro agente considere solo la recompensa inmediata.

En la fórmula anterior estamos asignando (←), o actualizando, el valor Q del estado actual y la acción del agente tomando primero un peso (1 − α) del antiguo valor Q y luego agregando el valor aprendido. El valor aprendido es una combinación de la recompensa por realizar la acción actual en el estado actual y la recompensa máxima descontada del siguiente estado en el que estaremos una vez que realicemos la acción actual.
Básicamente, estamos aprendiendo la acción adecuada a tomar en el estado actual al observar la recompensa por la combinación estado / acción actual y las recompensas máximas para el siguiente estado. Esto eventualmente hará que nuestro taxi considere la ruta con las mejores recompensas.
El valor Q de un par estado-acción es la suma de la recompensa instantánea y la recompensa futura descontada (del estado resultante). La forma en que almacenamos los valores Q para cada estado y acción es a través de una tabla Q

La tabla Q es una matriz donde tenemos una fila para cada estado (500) y una columna para cada acción (6). Primero se inicializa a 0 y luego los valores se actualizan después del entrenamiento. Tenga en cuenta que la Q-table tiene las mismas dimensiones que la mesa de recompensas, pero tiene un propósito completamente diferente.

En la siguiente figura los valores de la Q-Table se inicializan a cero y se van actualizando durante el aprendizaje.  Los valores optimizan el recorrido del agente a traves del entorno buscando las máximas recompensas. 

![image.png](attachment:image.png)

### Resumen del proceso de Q-Learning

    • Inicializar la tabla Q a todo ceros.
    • Comenzar a explorar acciones: para cada estado, seleccione cualquiera de las posibles acciones para el estado actual (S).
    • Ir al siguiente estado (S ') como resultado de esa acción (a).
    • Para todas las acciones posibles del estado (S '), seleccione la que tenga el valor Q más alto.
    • Actualizar los valores de la tabla Q utilizando la ecuación.
    • Establecer el siguiente estado como el estado actual.
    • Si se alcanza el estado objetivo, finalizar y repetir el proceso.

### Explotación de valores aprendidos

Después de una fase de exploración aleatoria de acciones, los valores Q tienden a converger sirviendo a nuestro agente como una función de valor de acción que puede explotar para elegir la acción mejor para un estado dado.
Existe una compensación entre exploración (elegir una acción aleatoria) y explotación (elegir acciones basadas en valores Q ya aprendidos). Queremos evitar que la acción siga siempre la misma ruta y posiblemente se sobreajuste, por lo que introduciremos otro parámetro llamado ϵ "épsilon" para atender esto durante el entrenamiento.
En lugar de simplemente seleccionar la acción de valor Q mejor aprendida, a veces preferimos explorar más el espacio de acción. Un valor de épsilon más bajo da como resultado episodios con más penalizaciones (en promedio), lo cual es obvio porque estamos explorando y tomando decisiones al azar.

In [ ]:
# Primero se inicializa la Q-table a 500×6500×6 matrix of zeros:

import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])


Ejecutar el aprendizaje es un proceso lento pero se realiza una única vez. 
Una vez entrenado, podemos resolver cualquier problema de este entorno simplemente consultando la tabla y eligiendo la acción que maximiza la recompensa en cada paso (explotación). 
El algoritmo de entrenamiento que actualizará esta Q-table a medida que el agente explora el entorno durante miles de episodios. 

En la primera parte del bucle (while not done) decidimos si elegir una acción aleatoria o explotar los valores Q ya calculados. Esto se hace simplemente usando el valor épsilon y comparándolo con la función random.uniform (0, 1), que devuelve un número arbitrario entre 0 y 1.
Ejecutamos la acción elegida en el entorno para obtener el next_state y la recompensa por realizar la acción. Después de eso, calculamos el valor Q máximo para las acciones correspondientes al next_state, y con eso, podemos actualizar fácilmente nuestro valor Q al new_q_value:

In [ ]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")


Episode: 100000
Training finished.

Wall time: 26.9 s


In [ ]:
# La tabla Q ha cambiado despues de 100000 episodios. Vamos a ver cuales son los Q-values aprendidos en el estado de ejemplo. 
q_table[328]

array([ -2.41837066,  -2.27325184,  -2.41837066,  -2.3639511 ,
       -11.3639511 , -11.3639511 ])

El valor máximo de Q es "norte" (-1,971), por lo que parece que Q-learning ha aprendido efectivamente la mejor acción a realizar en el estado de la imagen.

### Evaluar el comportamiento del agente despues del proceso de Q-learning 

Para evaluar el comportamiento de nuestro agente no necesitamos explorar más acciones. En el comportamiento del agente ahora la siguiente acción siempre se selecciona utilizando el mejor valor Q:

In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")


Results after 100 episodes:
Average timesteps per episode: 13.15
Average penalties per episode: 0.0


Podemos ver en la evaluación que el comportamiento del agente ha mejorado significativamente y no hay penalizaciones, lo que significa que realizó las acciones correctas de recogida / devolución con 100 pasajeros diferentes.

Comparando los dos agentes vemos como aunque con Q-learning el agente comete errores inicialmente durante la exploración, una vez que ha explorado lo suficiente (visto la mayoría de los estados), puede actuar sabiamente maximizando las recompensas haciendo movimientos inteligentes. 

Veamos cuánto mejor es nuestra solución de Q-learning en comparación con el agente que realiza movimientos aleatorios.

Evaluamos a nuestros agentes de acuerdo con las siguientes métricas,

    • Número medio de penalizaciones por episodio: Cuanto menor sea el número, mejor será el desempeño de nuestro agente. Idealmente, nos gustaría que esta métrica fuera cero o muy cercana a cero.

    • Número promedio de pasos de tiempo por viaje: también queremos un número pequeño de pasos de tiempo por episodio, ya que queremos que nuestro agente dé pasos mínimos (es decir, el camino más corto) para llegar al destino.

    • Promedio de recompensas por movimiento: cuanto mayor sea la recompensa, significa que el agente está haciendo lo correcto. Es por eso que decidir las recompensas es una parte crucial del aprendizaje por refuerzo. En nuestro caso, dado que tanto los tiempos como las penalizaciones se recompensan negativamente, una recompensa promedio más alta significaría que el agente llega al destino lo más rápido posible con la menor cantidad de penalizaciones.


    Average rewards per move	-3.9012092102214075	0.6962843295638126
    Average number of penalties per episode	920.45	0.0
    Average number of timesteps per trip	2848.14	12.38

Estas métricas se calcularon en más de 100 episodios. 
Y como muestran los resultados, nuestro agente de Q-learning tiene un buen comportamiento. 


### Hiperparámetros y optimizaciones

Los valores de `alpha`,` gamma` y `epsilon` que hemos utilizado han sido elegidos por intuición, prueba y error pero hay mejores formas de obtener buenos valores. Idealmente, los tres deberían disminuir con el tiempo porque a medida que el agente continúa aprendiendo, en realidad construye antecedentes más válidos y duraderos;

    • α: (la tasa de aprendizaje) debería disminuir a medida que continúa adquiriendo una base de conocimientos cada vez mayor.
    • γ: a medida que se acerca cada vez más al valor límite, su preferencia por la recompensa a corto plazo debería aumentar, ya que no estará el tiempo suficiente para obtener la recompensa a largo plazo, lo que significa que su gamma debería disminuir.
    • ϵ: a medida que desarrollamos nuestra estrategia, tenemos menos necesidad de exploración y más explotación para obtener más utilidad de nuestra política, por lo que a medida que aumentan los ensayos, épsilon debería disminuir.

Una forma de obtener la combinación correcta de valores de hiperparámetros sería usar optimización local con algoritmos genéticos. 

Q-learning es uno de los algoritmos de aprendizaje por refuerzo más fáciles. Sin embargo, el problema con la obtención de Q es que, una vez que el número de estados en el entorno es muy alto, se vuelve difícil implementarlos con la tabla Q, ya que el tamaño sería muy, muy grande. Por eso se utilizan redes neuronales profundas en lugar de Q-table (Deep Reinforcement Learning). La red neuronal recibe información de estado y acciones en la capa de entrada y aprende a generar la acción correcta a lo largo del tiempo. Las técnicas de aprendizaje profundo (como las redes neuronales convolucionales) también se utilizan para interpretar los píxeles en la pantalla y extraer información del juego (como puntuaciones), y luego dejar que el agente controle el juego.

### Ejercicios   

#### Ejercicio 1. 
Hemos realizado aprendizaje. Utiliza los valores Q aprendidos para solucionar otro problema, es decir, cambiando el estado inicial/objetivo  


#### Ejercicio 2.

En el problema del Taxi resuelto con Q-Learning se pide experimentar con distintos estados iniciales y distintos valores de los hiperparámetros y con distinto números de episodios de aprendizaje y comentar los resultados obtenidos. Observa el comportamiento del agente con los valores límite de los hiperparámetros.

Comenta de forma razonada las conclusiones obtenidas de los distintos procesos de aprendizaje. 
Evalua y compara los agentes respecto a las métricas dadas 

#### Ejercicio 3.

Razona cómo se comportaría el agente con Q-learning si lo comparamos con un agente que resuelve el problema con búsqueda en el espacio de estados. Indica las ventajas e inconvenientes de las dos aproximaciones. 



In [ ]:
# Ejercicio 1

estado = env.encode(2, 2, 3, 1)
env.s = estado

total_epochs, total_penalties = 0, 0
epochs, penalties, reward = 0, 0, 0
done = False

while not done:
    action = np.argmax(q_table[estado])
    estado, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"penalties: {total_penalties}")
print(f"timesteps: {total_epochs}")

penalties: 0
timesteps: 55


In [ ]:
# Ejercicio 2

In [ ]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [ ]:
%%time
"""Training the agent"""

state = env.encode(2, 2, 3, 1)
env.s = estado

q_table = np.zeros([env.observation_space.n, env.action_space.n])

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 1001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 1000
Training finished.

Wall time: 43.6 s


In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 50000

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")


Results after 50000 episodes:
Average timesteps per episode: 13.0669
Average penalties per episode: 0.0


     Probando con 50 000 episodios

    Episodes   alpha    gamma   epsilon  avg timesteps  avg penalties
    100 000     0.1      0.6     0.1         13.149         0.0
    50 000      0.1      0.6     0.1         13.218         0.0
    
    1 000       0.1      0.6     1           13.132         0.0
    500         0.1      0.6     0.1         13.047         0.0
    
    1 000 000   0.3      0.6     0.1         13.011         0.0
    100 000     0.3      0.6     0.1         12.975         0.0
    50 000      0.3      0.6     0.1         13.109         0.0
    
    100 000     0.7      0.6     0.1         13.042         0.0
    50 000      0.7      0.6     0.1         13.069         0.0
    
    100 000     0.3      0.8     0.1         13.149         0.0
    50 000      0.3      0.8     0.1         13.005         0.0
    
    100 000     0.3      0.6     0.3         13.142         0.0
    50 000      0.3      0.6     0.3         12.932         0.0
    
    50 000      0.8      0.3     0.9         13.115         0.0
    10 000      0.8      0.3     0.9         13.039         0.0
    1 000       0.8      0.3     0.9         13.165         0.0
    
    

CONCLUSIONES 

- Queda muy parecido, seguramente por un número de episodios de prueba insuficiente.

In [ ]:
# Ejercicio 3

    
 - El agente con Q-learning resuelve cada episodio con mucha mayor velocidad y eficacia una vez que está bien entrenado. Sin embargo, el tiempo de entrenamiento puede llegar a ser prolongado y se debe estudiar qué valores para los hiperparámetros alpha gamma y épsilon son adecuados. Por otro lado, se tiene la desventaja de que si se cambia el problema se deberá calcular la tabla q de nuevo, lo que lleva asociado un elevado coste en tiempo.
 - Por el contrario, si se usa un agente de búsqueda en el espacio de soluciones, este será más ágil a la hora de realizar cambios en el problema pues cada cálculo es independiente. Como contrapartida se tiene que se recorren más estados y el tiempo de resolución de cada caso es mayor en general.
 - En conclusión, de forma general será más efectivo usar un agente con Q-learning cuando se vayan a resolver muchos casos de un mismo problema porque amortizaremos el gasto en tiempo de calcular la tabla q y preferiremos usar un agente de búsqueda en caso contrario.

### Parte 2. Cart Pole

Realizar el mismo proceso de entrenamiento con QLearning para otro entorno de OpenAI
https://gym.openai.com/envs/CartPole-v1/  

Como lo has usado en el ejercicio anterior no necesitas volver a instalar la librería Gym que ya incluye el entorno CartPole.

In [ ]:
## pip install gym[all] 
## para instalar gym con todas sus dependencias

Un poste está unido por una articulación no accionada a un carro, que se mueve a lo largo de una pista sin fricción. El sistema se controla aplicando una fuerza de +1 o -1 al carro. El péndulo comienza en posición vertical y el objetivo es evitar que se caiga. Se proporciona una recompensa de +1 por cada paso de tiempo que el poste permanece en posición vertical. El episodio termina cuando el poste está a más de 15 grados de la vertical o el carro se mueve más de 2.4 unidades desde el centro.

In [ ]:
# Generamos el entorno CartPole v1 y realizamos acciones aleatorias 

env = gym.make('CartPole-v1')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

ImportError: 
    Cannot import pyglet.
    HINT: you can install pyglet directly via 'pip install pyglet'.
    But if you really just want to install all Gym dependencies and not have to think about it,
    'pip install -e .[all]' or 'pip install gym[all]' will do it.
    

### Ejercicios.

#### Ejercicio 4. 
Se pide realizar un agente que aprenda a resolver el problema del CartPole usando Q-Learning. Comenta el resultado obtenido y realiza pruebas (como en el ejercicio 2) para comprobar el comportamiento con distintos valores de los hiperparámetros. 

En el ejercicio 4 el entorno propuesto es Cart Pole pero puedes usar cualquiera de los incluidos en OPEN AI, por ejemplo, también es sencillo el entorno FrozenLake-v0  https://gym.openai.com/envs/FrozenLake-v0/ 
En este entorno el agente controla el movimiento de un personaje en un mundo de rejilla. Algunas baldosas son transitables (walkable) y otras hacen que el agente caiga al agua. La dirección de movimiento del agente es incierta y solo depende parcialmente de la dirección elegida. 
La recompensa se obtiene cuando el agente llega a traves de un camino transitable a una casilla objetivo. 

In [ ]:
env = gym.make('FrozenLake-v1')
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
4
  (Down)
SFFF
FHFH
FFFH
HFFG
8
Episode finished after 3 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
1
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Down)
SFFF
FHFH
FFFH
HFFG
2
  (Left)
SFFF
FHFH
FFFH
HFFG
2
  (Up)
SFFF
FHFH
FFFH
HFFG
3
  (Left)
SFFF
FHFH
FFFH
HFFG
2
  (Right)
SFFF
FHFH
FFFH
HFFG
3
  (Down)
SFFF
FHFH
FFFH
HFFG
3
Episode finished after 23 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Left)
SFFF
FHFH
FFFH
HFFG
4
Episode finished after 3 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (U

In [ ]:
%%time
"""Training the agent"""

q_table = np.zeros([env.observation_space.n, env.action_space.n])

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.3
gamma = 0.9
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_success = []

for i in range(1, 10001):
    state = env.reset()

    epochs, success, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == 1:
            success += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 10000
Training finished.

Wall time: 3.51 s


In [ ]:
total_epochs, total_success = 0, 0
episodes = 100000

for _ in range(episodes):
    state = env.reset()
    epochs, success, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == 1:
            success += 1

        epochs += 1

    total_success += success
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average success per episode: {total_success / episodes}")


Results after 100000 episodes:
Average timesteps per episode: 26.91776
Average success per episode: 0.38782


            Probando con 100 000 episodios

     Comb.  Episodes   alpha    gamma   epsilon  sucess_rate (0-1)

         1  1 000      0.1      0.6     0.1      0.0
            10 000     0.1      0.6     0.1      0.119
            100 000    0.1      0.6     0.1      0.167

         2  10 000     0.1      0.2     0.1      0.055
            50 000     0.1      0.2     0.1      0.108
            100 000    0.1      0.2     0.1      0.108

         3  10 000     0.9      0.6     0.1      0.101
            100 000    0.9      0.6     0.1      0.146

         4  10 000     0.1      0.6     0.9      0.567 Volviendo a entrenar da 0.138
            100 000    0.1      0.6     0.9      0.116

         5  100 000    0.1      0.6     1.0      0.251 Volviendo a entrenar da 0.345

         6  10 000     0.1      0.9     0.1      0.0   Volviendo a entrenar 0.601, 0.0 y 0.505(??)
            100 000    0.1      0.9     0.1      0.203 Volviendo a entrenar 0.711 (QUÉ LOCURA)

         7  10 000     0.5      0.9     0.1      0.0   Otra vez da lo mismo y otra da 0.143  
            100 000    0.5      0.9     0.1      0.510

         8  10 000     0.3      0.9     0.1      0.0   Otra vez igual y otra vez 0.388
            50 000     0.3      0.9     0.1      0.253
            100 000    0.3      0.9     0.1      0.580
            1 000 000  0.3      0.9     0.1      0.448

         9  100 000    0.4      0.9     0.1      0.443

         10 50 000     0.3      0.9     0.4      0.132
            100 000    0.3      0.9     0.4      0.328

CONCLUSIONES:

- Lo primero que podemos observar para la mayoría de combincaciones de hiperparámetros es que un mayor número de episodios en la fase de entrenamiento resulta por lo general en un mayor porcentaje de éxito a la hora de ejecutar el algoritmo.

- Otra observación general es que valores extremos de cualquiera de los parámetros (cercanos a 0 o a 1) no suelen dar buenos resultados. Esto lo observamos en las 6 primeras combianciones, en las que por lo general no se pasa de un 25% de éxito.

- Parámetro Gamma (tasa de descuento): Debido a que es un problema que solo recibe una recompensa y además positiva cuando finaliza con éxito, supusimos que tener "más en cuenta" la recompensa a largo plazo que la inmediata (valor de gamma alto) positivo daría buenos resultados. Podemos observar que en combinaciones de parámetros en la que la gamma es más cercana a 0 tenemos un porcentaje de éxito más bajo (comb.2 gamma=0.2 éxito=10.8% con 100 000 episodios) que en otras con la gamma más cercana a 1 (comb.1 gamma=0.6 éxito=16.7% con 100 000 episodios, ó comb.6 gamma=0.9 éxito= 20.3%).

- Parámetro Epsilon (Aleatoriedad): Con valores de epsilon cercanos a 1 como en las combinaciones 4 (epsilon=0.9) y 5 (epsilon=1) podemos observar como para entrenamientos de 100 000 episodios obtenemos porcentajes de éxitos bajos (11.6% y 25.1%) ya que no explota lo aprendido nunca, sino que genera aleatoriamente el 100% de las veces. Se pueden dar casos en los que el porcentaje de éxito sea alto ya que las decisiones que va a tomar van a ser basadas en la aleatoriedad y puede resultar que se haya generado aleatoriamente una tabla-q "más o menos" buena, pero en general no es el caso. También es importante apuntar que al ser 100% aleatorio da igual el número de veces que entrenemos, (lo que se puede observar en la combinación 4 de parámetros, nos da una tasa de éxito mucho mayor entrenando menos, lo cual es casualidad, resultado de la aleatoriedad).

- Parámetro alpha (tasa de aprendizaje): Como se indica al principio observamos que valores extremos dan un porcentaje de éxito menor que valores más intermedios (como en la últimas combinaciones).

- Hemos observado que en general da mejores resultados una combinación de una tasa e apredizaje intermedia combinada con una tasa de descuento alta y un epsilon bajo. Es decir, conviene tener en cuenta en la misma medida (más o menos) la nueva experiencia que lo ya aprendido hasta el momento, así como tener en cuenta una recompensa mixta en la que se le da más prioridad a la recompensa a largo plazo que a los refuerzos inmediatos pues en la mayoría de pasos estos últimos no existen. La aleatoridad es necesaria para poder explorar el espacio de soluciones pero no en medida desproporcionada.



#### Ejercicio 5.  Opcional.   

Aplica el algoritmo Q-learning para diseñar un agente que aprenda a resolver alguno de los puzles de la práctica 1 (puzle de 8, jarras, misioneros,..) y comenta el resultado comparandolo con el agente que ya tienes hecho de la práctica anterior que resuelve el problema usando búsqueda en espacio de estados.  
Discute claramente las ventajas e inconvenientes con las métricas y resultados obtenidos.